In [1]:
using Flux
using Flux: onehotbatch, argmax, crossentropy, throttle
using Base.Iterators: repeated, partition
using MNIST
using CLArrays

In [76]:
X_train, y_train = traindata()
X_test, y_test = testdata()

# X_train = [CLArray(Float32.(reshape(X_train[:, i], (28, 28)))) for i in 1:60_000 ]
# X_test = [CLArray(Float32.(reshape(X_test[:, i], (28, 28)))) for i in 1:10_000 ]

X_train = [(reshape(X_train[:, i], (28, 28))) for i in 1:60_000 ]
X_test = [(reshape(X_test[:, i], (28, 28))) for i in 1:10_000 ]

# y_train = CLArray(Float32.(onehotbatch(y_train, 0:9)))
# y_test = CLArray(Float32.(onehotbatch(y_test, 0:9)))

y_train =  onehotbatch(y_train, 0:9)
y_test = onehotbatch(y_test, 0:9)

println(size(X_train))
println(typeof(X_train))
println(size(y_train))
println(typeof(y_train))
println()
println(size(X_test))
println(typeof(X_test))
println(size(y_test))
println(typeof(y_test))

Stacktrace:
 [1] depwarn(::String, ::Symbol) at ./deprecated.jl:70
 [2] Array(::Type{Float64}, ::Int64, ::Int64) at ./deprecated.jl:57
 [3] traindata() at /Users/hitmann47/.julia/v0.6/MNIST/src/MNIST.jl:88
 [4] include_string(::String, ::String) at ./loading.jl:522
 [5] include_string(::Module, ::String, ::String) at /Users/hitmann47/.julia/v0.6/Compat/src/Compat.jl:174
 [6] execute_request(::ZMQ.Socket, ::IJulia.Msg) at /Users/hitmann47/.julia/v0.6/IJulia/src/execute_request.jl:154
 [7] (::Compat.#inner#16{Array{Any,1},IJulia.#execute_request,Tuple{ZMQ.Socket,IJulia.Msg}})() at /Users/hitmann47/.julia/v0.6/Compat/src/Compat.jl:496
 [8] eventloop(::ZMQ.Socket) at /Users/hitmann47/.julia/v0.6/IJulia/src/eventloop.jl:8
 [9] (::IJulia.##14#17)() at ./task.jl:335
while loading In[76], in expression starting on line 1
Stacktrace:
 [1] depwarn(::String, ::Symbol) at ./deprecated.jl:70
 [2] Array(::Type{Float64}, ::Int64) at ./deprecated.jl:57
 [3] traindata() at /Users/hitmann47/.julia/v0.6/

(60000,)
Array{Array{Float64,2},1}
(10, 60000)
Flux.OneHotMatrix{Array{Flux.OneHotVector,1}}

(10000,)
Array{Array{Float64,2},1}
(10, 10000)
Flux.OneHotMatrix{Array{Flux.OneHotVector,1}}


In [ ]:
X_test = cat(4, X_test...)

In [95]:
size(X_test)

(28, 28, 1, 10000)

In [100]:
for (idx, batch) in enumerate(partition(1:60000, 100))
#     println(typeof(batch),'\t', length(batch),'\t', batch[1],'\t', batch[end])
    println("Batch $idx")
    batch_train = [(cat(4, X_train[batch]...), y_train[:, batch])]
    
    Flux.train!(loss, batch_train, optimizer)
    
    if idx % 20 == 0
        println("Batch $idx")
        println("Test loss: $(loss(X_test, y_test)), acc: $(accuracy(X_test[:, :, :, 1:5000], y_test[:, 1:5000]))")
        println()
    end
    
end

Batch 1
Batch 2
Batch 3
Batch 4
Batch 5
Batch 6
Batch 7
Batch 8
Batch 9
Batch 10
Batch 11
Batch 12
Batch 13
Batch 14
Batch 15
Batch 16
Batch 17
Batch 18
Batch 19
Batch 20
Batch 20
Test loss: param(0.436978), acc: 0.8364

Batch 21
Batch 22
Batch 23
Batch 24
Batch 25
Batch 26
Batch 27
Batch 28
Batch 29
Batch 30
Batch 31
Batch 32
Batch 33
Batch 34
Batch 35
Batch 36
Batch 37
Batch 38
Batch 39
Batch 40
Batch 40
Test loss: param(0.250832), acc: 0.9018

Batch 41
Batch 42
Batch 43
Batch 44
Batch 45
Batch 46
Batch 47
Batch 48
Batch 49
Batch 50
Batch 51
Batch 52
Batch 53
Batch 54
Batch 55
Batch 56
Batch 57
Batch 58
Batch 59
Batch 60
Batch 60
Test loss: param(0.188544), acc: 0.9188

Batch 61
Batch 62
Batch 63
Batch 64
Batch 65
Batch 66
Batch 67
Batch 68
Batch 69
Batch 70
Batch 71
Batch 72
Batch 73
Batch 74
Batch 75
Batch 76
Batch 77
Batch 78
Batch 79
Batch 80
Batch 80
Test loss: param(0.148691), acc: 0.936

Batch 81
Batch 82
Batch 83
Batch 84
Batch 85
Batch 86
Batch 87
Batch 88
Batch 89
Batch 90


LoadError: [91mInterruptException:[39m

In [ ]:
# Partition into batches of size 100
# training_data = [(cat(4, float.(X_train[i])...), y_train[:,i])
#          for i in partition(1:60_000, 100)]

# X_train = cat(4, float.(X_train)...)


In [97]:
model = Chain(
            #28 * 28
          Conv2D((3, 3), 1=>32, relu),
            #26 * 26 * 32   
          x -> maxpool2d(x, 2),
            #13 * 13 * 32   
          Conv2D((3, 3), 32=>64, relu),
            #11 * 11 * 64
          x -> maxpool2d(x, 2),
            #5 * 5 * 64
          x -> reshape(x, :, size(x, 4)),
          Dense(1600, 128, relu),
          Dense(128, 10), 
          softmax)

Chain(Conv2D((3, 3), 1=>32, NNlib.relu), #99, Conv2D((3, 3), 32=>64, NNlib.relu), #100, #101, Dense(1600, 128, NNlib.relu), Dense(128, 10), NNlib.softmax)

In [98]:
# Use cross entropy loss here
loss(x, y) = crossentropy(model(x), y)
accuracy(x, y) = mean(argmax(model(x)) .== argmax(y))

accuracy (generic function with 1 method)

In [99]:
# Callback during the training
function eval_cb()
    
#     println("Training loss: $(loss(X_train, y_train)), acc: $(accuracy(X_train, y_train))")
    println("Test loss: $(loss(X_test, y_test)), acc: $(accuracy(X_test, y_test))")
    println()
end

optimizer = ADAM(params(model))

# Flux.train!(loss, training_data, optimizer, cb = throttle(eval_cb, 10))

(::#71) (generic function with 1 method)

In [ ]:
println("Optimization Finished!")
@printf "Model accuracy on test set: %.3f\n" accuracy(X_test, y_test)